<a href="https://colab.research.google.com/github/vindruid/face-detection-exploration/blob/master/comparison%20mmod%20and%20dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from IPython.display import HTML
from base64 import b64encode
import dlib
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from base64 import b64encode

In [0]:
# google drive download utils

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

# Download Model

### MMOD Model

source: https://github.com/davisking/dlib-models/

In [0]:
!wget https://github.com/davisking/dlib-models/raw/master/mmod_human_face_detector.dat.bz2

extract bz2

In [0]:
!bzip2 -d mmod_human_face_detector.dat.bz2

### caffe model

In [0]:
# https://drive.google.com/file/d/1gW1Hgk_KbXWAFvFQrnaZczZyFQJztzNC/view?usp=sharing
file_id = '1gW1Hgk_KbXWAFvFQrnaZczZyFQJztzNC'
destination = 'deploy.prototxt'
download_file_from_google_drive(file_id, destination)

In [0]:
# https://drive.google.com/file/d/1fYzYr7qFksB9ajPee4gKLEvB8pBy40i-/view?usp=sharing
file_id = '1fYzYr7qFksB9ajPee4gKLEvB8pBy40i-'
destination = 'res10_300x300_ssd_iter_140000.caffemodel'
download_file_from_google_drive(file_id, destination)

# Download Video & Show

In [0]:
# https://drive.google.com/file/d/1LGOPQ1vlmKW_4J8Z1BC8lYEOxeb_-RPG/view?usp=sharing
file_id = '1LGOPQ1vlmKW_4J8Z1BC8lYEOxeb_-RPG'
destination = 'girl open eyes.mp4'
download_file_from_google_drive(file_id, destination)

In [0]:
# Show video
mp4 = open(destination,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Video Properties

In [0]:
path_video = 'girl open eyes.mp4'

cap  = cv2.VideoCapture(path_video)
_, img = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

count_frame = 0

while img is not None: 
    _, img = cap.read()
    count_frame += 1

print("VIDEO FPS:", fps)
print("VIDEO width:", w)
print("VIDEO heigh:", h)
print("VIDEO frame:", count_frame)
print("VIDEO length:", (count_frame / fps), "second")


# Detect Face MMOD

In [0]:
dnnFaceDetector = dlib.cnn_face_detection_model_v1("./mmod_human_face_detector.dat")

In [0]:
%%time 
save_path = 'result_mmod_.mp4'
input_size = (200, 150) # w,h

cap  = cv2.VideoCapture(path_video)
_, img = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
w_origin = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h_origin = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w, h = input_size
w_ratio = w_origin/w
h_ratio = h_origin/h

vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (w_origin, h_origin))


while img is not None: 
    img_resized = cv2.resize(img, input_size)
    faceRects = dnnFaceDetector(img_resized, 0)
    for faceRect in faceRects:
        x1 = int(faceRect.rect.left() * w_ratio)
        y1 = int(faceRect.rect.top() * h_ratio)
        x2 = int(faceRect.rect.right() * w_ratio)
        y2 = int(faceRect.rect.bottom() * h_ratio)

        cv2.rectangle(img, (x1, y1), (x2,y2), color = (255, 0, 0), thickness=2 )

    vid_writer.write(img)
    _, img = cap.read()

vid_writer.release()


In [0]:
# compress video
result_path = 'result_mmod.mp4'
os.system(f"ffmpeg -i {save_path} -vcodec libx264 {result_path}")

!rm {save_path}

In [0]:
# Show video
mp4 = open(result_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Detect Face caffe

In [0]:
net = cv2.dnn.readNetFromCaffe('deploy.prototxt',
                               'res10_300x300_ssd_iter_140000.caffemodel')

In [0]:
%%time 
save_path = 'result_caffe_.mp4'

cap  = cv2.VideoCapture(path_video)
_, img = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
w_origin = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h_origin = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (w_origin, h_origin))

while img is not None: 

    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
	                              (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.8:
            box = detections[0, 0, i, 3:7] * np.array([w_origin, h_origin, w_origin, h_origin])
            (x1, y1, x2, y2) = box.astype("int")

            cv2.rectangle(img, (x1, y1), (x2,y2), color = (0, 255, 0), thickness=2 )

    vid_writer.write(img)

    _, img = cap.read()

vid_writer.release()


In [0]:
# compress video
result_path = 'result_caffe.mp4'
os.system(f"ffmpeg -i {save_path} -vcodec libx264 {result_path}")

!rm {save_path}

In [0]:
# Show video
mp4 = open(result_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)